<a href="https://colab.research.google.com/github/ccarpenterg/LearningPyTorch1.x/blob/master/06_cifar_10_fine_tuning_pretrained_convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CIFAR-10: Fine-tuning a pre-trained convnet

In [0]:
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torchvision import models
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

In [0]:
print('PyTorch version:', torch.__version__)
print('Torchvision version:', torchvision.__version__)

In [0]:
dataset_transform = transforms.Compose([
    transforms.ToTensor()
])

train_set = CIFAR10('./cifar10', train=True, download=True, transform=dataset_transform)
valid_set = CIFAR10('./cifar10', train=False, download=True, transform=dataset_transform)

In [0]:
train_loader = DataLoader(train_set, batch_size=128, num_workers=0, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=512, num_workers=0, shuffle=False)

In [0]:
def train(model, loss_fn, optimizer):
    model.train()
    
    train_batch_losses = []
    
    for batch, labels in train_loader:
        batch = batch.to(cuda)
        labels = labels.to(cuda)
        
        optimizer.zero_grad()
        y_pred = model(batch)
        loss = loss_fn(y_pred, labels)
        loss.backward()
        optimizer.step()
        
        train_batch_losses.append(float(loss))
        
        mean_loss = statistics.mean(train_batch_losses)
        
    return mean_loss

def validate(model, loss_fn, optimizer):
    model.eval()
    
    predictions = []
    
    with torch.no_grad():
        validation_batch_losses = []
        
        for batch, labels in valid_loader:
            batch = batch.to(cuda)
            labels = labels.to(cuda)
            
            labels_pred = model(batch)
            loss = loss_fn(labels_pred, labels)
            
            validation_batch_losses.append(float(loss))
            
            mean_loss = statistics.mean(validation_batch_losses)
            
    return mean_loss

def accuracy(model, loader):
    correct = 0
    total = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch, labels in loader:
            batch = batch.to(cuda)
            labels = labels.to(cuda)
        
            labels_pred = model(batch)
        
            _, predicted = torch.max(labels_pred.data, 1)
        
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    return (100 * correct / total)

In [0]:
model = models.resnet18(pretrained=True)

# freezing the resnet by setting requires_grad to False for all parameters
for parameter in model.parameters():
    parameter.requires_grad = False

# we need to know the number of features that are fed to the last dense layer
num_features = model.fc.in_features

# now we remove the ResNet18's classifier and replace it with a 10 classes classifier
model.fc = nn.Linear(num_features, 10)